In [9]:
#import all libraries and create variables


from web3 import Web3
from dotenv import load_dotenv
import os
import json
from requests import get
load_dotenv()

w3 = Web3(Web3.HTTPProvider('https://black-ultra-reel.discover.quiknode.pro/0f2c80809714e75ffb187f225f3a1613b58862a8/'))
address1 = '0x34a0216c5057bc18e5d34d4405284564efd759b2'
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')
BASE_URL = "https://api.etherscan.io/api"

list_of_hashes = []


#function to easily make the apicall url

def make_api_URL(module, action, address, **kwargs):
    url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={ETHERSCAN_API_KEY}"
    
    for key, value in kwargs.items():
        url += f"&{key}={value}"
        
    return url


#function that attempts to get the swap transaction data from topic0, uniswap trades. 

def get_swaps(address):
    get_swaps_url = make_api_URL('logs', 'getLogs', address1, topic0='0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822', startblock=0, endblock=99999999, page=1, offset=10000, sort='desc') 
    response = get(get_swaps_url)
    data=response.json()['result']
    
    #topic0 data is returned and displayed at value 'result', which is a list of dictionaries. 
    #we need to iterate through this list of dictionaries and return all the values that are equal to "transactionHash"
    #those values are appended to a new list where they can easily be parsed through. 
    
    for dictionary in data:
        for key in dictionary:
            if key =='transactionHash':
                list_of_hashes.append(dictionary[key])
                
                
    #Now we iterate through the list of transaction hashes, and use the ABI endpoint to try and decode the data. 
    #something is missing here. It returns some data that makes sense, but some values are throwing it an error. It's 4am right now. I'll try to fix it later, if you guys take a look at it, this is where I'm stuck. 
    for tx_hash in list_of_hashes:
        tx = w3.eth.get_transaction(tx_hash)
        abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={tx['to']}&apikey={ETHERSCAN_API_KEY}"
        abi = json.loads(requests.get(abi_endpoint).text)
        contract = w3.eth.contract(address=tx["to"], abi=abi["result"])
        func_obj, func_params = contract.decode_function_input(tx["input"])
        
        print(func_params)
    
    
    
    




get_swaps(address1)

{'amountIn': 25000000000000000000, 'amountOutMin': 14328807361857257, 'path': ['0x6B175474E89094C44Da98b954EedeAC495271d0F', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', '0x261EfCdD24CeA98652B9700800a13DfBca4103fF'], 'to': '0x8Cc8Aa103bfE1CD8755DC80C64539370B587909F', 'deadline': 1590748527}
{'fromToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'toToken': '0x261EfCdD24CeA98652B9700800a13DfBca4103fF', 'fromTokenAmount': 2500000000000000000, 'minReturnAmount': 308322978144010633, 'guaranteedAmount': 311437351660616802, 'referrer': '0x0000000000000000000000000000000000000000', 'callAddresses': ['0x5Be5b30A720093C9d1B1E6CF8441280a48a495F8'], 'callDataConcat': b'\xf8\x83\t\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00&\x1e\xfc\xdd$\xce\xa9\x86R\xb9p\x08\x00\xa1=\xfb\xcaA\x03\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

TypeError: __init__() missing 2 required positional arguments: 'doc' and 'pos'

AttributeDict({'baseFeePerGas': 8045317912,
 'difficulty': 12207864436779337,
 'extraData': HexBytes('0x706f6f6c696e2e636f6d1d7d1a11bb009d999e'),
 'gasLimit': 29999914,
 'gasUsed': 2129727,
 'hash': HexBytes('0x7104f40c7bb96c4683681c5b324f9bc170243fa432292c5ce0ba23b97e73cf75'),
 'logsBloom': HexBytes('0x020400400000000000010000000040100000010020000000002000000000000000000000800000000000020000000100000000000000000000000000202000008000000000000000080000080000200040802000000000400000000100008800000000000200200200007040000008000500006400000000000000100000000000000100000010010000000000000002000000400120004020000000001000000280000040082000000000c0000000080000000000000400000900000000004001000002040000000300000080800000002000822000000000000000001020214010002000100020400010000000000004400000000000800000000020800800'),
 'miner': '0xCD458d7F11023556cC9058F729831a038Cb8Df9c',
 'mixHash': HexBytes('0x75cd69ee6d8fb626c72331366b1dd00b29a302a3b134d40fc9233e86b70a4490'),
 'nonce': HexBytes('0x001000002